In [1]:
#没增加维度，能运行

In [2]:
import random
import torch
from sklearn.model_selection import KFold
import math
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd


In [3]:
import torch

# 设置GPU使用方式
# 获取GPU列表
gpus = [torch.cuda.device(i) for i in range(torch.cuda.device_count())]

if gpus:
    try:
        # 设置每个GPU为增长式占用
        for gpu in gpus:
            torch.cuda.set_per_process_memory_fraction(1.0, gpu)  # 可根据需要调整占用比例
    except RuntimeError as e:
        # 打印异常
        print(e)


In [4]:

import torch
from torch import nn


class Net(nn.Module):
    def __init__(self, n_chans,n_classes):
        super(Net, self).__init__()

        self.temp_conv1 = nn.Conv1d(n_chans, n_chans, kernel_size=2, stride=1 ,groups=n_chans)
        self.temp_conv2 = nn.Conv1d(n_chans, n_chans, kernel_size=2, stride=1 ,groups=n_chans)
        self.temp_conv3 = nn.Conv1d(n_chans, n_chans, kernel_size=2, stride=1 ,groups=n_chans)
        self.temp_conv4 = nn.Conv1d(n_chans, n_chans, kernel_size=2, stride=1 ,groups=n_chans)
        self.temp_conv5 = nn.Conv1d(n_chans, n_chans, kernel_size=2, stride=1 ,groups=n_chans)
        self.temp_conv6 = nn.Conv1d(n_chans, n_chans, kernel_size=2, stride=1 ,groups=n_chans)

        self.chpool1    = nn.Sequential(
            nn.Conv1d(n_chans, 32, kernel_size=2,groups=1),
            nn.BatchNorm1d(32),
            nn.ELU(),
            nn.Conv1d(32, 32, kernel_size=2,groups=1),
            nn.BatchNorm1d(32),
            nn.ELU())

        self.chpool2    = nn.Sequential(
            nn.Conv1d(n_chans, 32, kernel_size=2,groups=1),
            nn.BatchNorm1d(32),
            nn.ELU(),
            nn.Conv1d(32, 32, kernel_size=2,groups=1),
            nn.BatchNorm1d(32),
            nn.ELU()
           )

        self.chpool3    = nn.Sequential(
            nn.Conv1d(n_chans, 32, kernel_size=2,groups=1),
            nn.BatchNorm1d(32),
            nn.ELU(),
            nn.Conv1d(32, 32, kernel_size=2,groups=1),
            nn.BatchNorm1d(32),
            nn.ELU(),
            nn.Dropout(0.5)
            )
            
        self.chpool4    = nn.Sequential(
            nn.Conv1d(n_chans, 32, kernel_size=2,groups=1),
            nn.BatchNorm1d(32),
            nn.ELU(),
            nn.Conv1d(32, 32, kernel_size=2,groups=1),
            nn.BatchNorm1d(32),
            nn.ELU(),
            nn.Dropout(0.5)
            )

        self.chpool5    = nn.Sequential(
            nn.Conv1d(n_chans, 32, kernel_size=2,groups=1),
            nn.BatchNorm1d(32),
            nn.ELU(),
            nn.Conv1d(32, 32, kernel_size=2,groups=1),
            nn.BatchNorm1d(32),
            nn.ELU(),
            nn.Dropout(0.5)
            )

        self.classifier = nn.Sequential(
            nn.Linear(160,64),
            nn.ELU(),
            nn.Linear(64,32),
            nn.ELU(),
            nn.Linear(32,n_classes))
        

    def forward(self, x , training=True):

        temp_x  = self.temp_conv1(x)               
        temp_w1 = self.temp_conv2(temp_x)         
        temp_w2 = self.temp_conv3(temp_w1)      
        temp_w3 = self.temp_conv4(temp_w2)       
        temp_w4 = self.temp_conv5(temp_w3)      
        temp_w5 = self.temp_conv6(temp_w4)      

        w1      = self.chpool1(temp_w1).mean(dim=(-1))
        w2      = self.chpool2(temp_w2).mean(dim=(-1))
        w3      = self.chpool3(temp_w3).mean(dim=(-1))
        w4      = self.chpool4(temp_w4).mean(dim=(-1))
        w5      = self.chpool5(temp_w5).mean(dim=(-1))
       
    
        concat_vector  = torch.cat([w1,w2,w3,w4,w5],1)
        #classes        = nn.functional.log_softmax(self.classifier(concat_vector),dim=1)  
        classes = self.classifier(concat_vector)
        return classes

In [5]:
#导入数据
#rest1
datapath1=r'D:\JQ_YJS\飞行试验数据\处理后\rest1.npy' 
data1=np.load(datapath1)
#print(data1.shape)
'''data1=torch.tensor(data1).unsqueeze(1)
print(data1.shape)
data1=data1.numpy()'''

#rest2
datapath2=r'D:\JQ_YJS\飞行试验数据\处理后\rest2.npy'
data2=np.load(datapath2)
#print(data2.shape)
'''data2=torch.tensor(data2).unsqueeze(1)
print(data2.shape)
data2=data2.numpy()'''
data_all=np.concatenate((data1,data2),axis=0)
#标签制作
label_all = torch.cat([torch.zeros(6344), torch.ones(6589)]).long()  # 标签：前10505个为0，后10505个为1
print(label_all.shape)
print(data_all.shape)





torch.Size([12933])
(12933, 30, 200)


In [6]:
# 创建十折交叉验证
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

# 保存结果的列表
historys = []
test_pred = []
test_real = []
accuracy, precision, recall, f1score = [], [], [], []
batchsz = 16
num_epochs = 100

# 进行十折交叉验证
for fold, (train_ind, test_ind) in enumerate(kfold.split(data_all, label_all)):
    print('fold号:', fold + 1)

   

    # 每一折验证前都要打乱训练集样本顺序
    n = len(train_ind)
    A = np.linspace(0, n - 1, n, dtype=int)
    random.shuffle(A)

    # 构建训练集、验证集、测试集
    epoch_train = data_all[train_ind[A[:int(0.8 * n)]]]
    epoch_val = data_all[train_ind[A[int(0.8 * n):]]]
    epoch_test = data_all[test_ind]
    label_train = label_all[train_ind[A[:int(0.8 * n)]]]
    label_val = label_all[train_ind[A[int(0.8 * n):]]]
    label_test = label_all[test_ind]

    # 转换为Tensor并创建DataLoader
    train_dataset = TensorDataset(torch.tensor(epoch_train, dtype=torch.float32), torch.tensor(label_train, dtype=torch.long))
    val_dataset = TensorDataset(torch.tensor(epoch_val, dtype=torch.float32), torch.tensor(label_val, dtype=torch.long))
    
    train_loader = DataLoader(train_dataset, batch_size=batchsz, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batchsz, shuffle=False)

    # 选择、创建模型
    model = Net(n_classes=1, n_chans=30)
    #print(model)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    # 配置模型训练
    criterion = nn.BCEWithLogitsLoss()  # 使用交叉熵损失
    
    optimizer = optim.SGD(model.parameters(), lr=1e-3)
    #optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

    # 开始训练模型
    print('开始训练!!')
    
    history = {'train_loss': [], 'val_loss': []}

    for epoch in range(num_epochs):  # 训练100个epoch
        model.train()
        running_loss = 0.0

        # 训练阶段
        for inputs, targets in train_loader:
            #print(f"训练批次输入数据形状: {inputs.shape}")
            inputs = inputs.to(device)
            
            targets = targets.to(device)
            optimizer.zero_grad()
            
            outputs = model(inputs)
            targets = targets.view(-1).float() 
            #targets=targets.view(-1,1).float()
            loss = criterion(outputs.view(-1), targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        history['train_loss'].append(avg_train_loss)

        #print('Epoch:{}/{}'.format(epoch+1,num_epochs))
        
        # 计算训练集的指标
        with torch.no_grad():
            train_pred = model(torch.tensor(epoch_train, dtype=torch.float32).to(device))
            train_pred = (torch.sigmoid(train_pred) > 0.5).cpu().numpy().astype(int)
            
            acc_train = accuracy_score(label_train, train_pred)
            pre_train = precision_score(label_train, train_pred, average='macro')
            rec_train = recall_score(label_train, train_pred, average='macro')
            f1_train = f1_score(label_train, train_pred, average='macro')

        # 输出训练集指标
       
        # 验证阶段
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                

                inputs = inputs.to(device)
            
                targets = targets.to(device)
                outputs = model(inputs)
                #targets=targets.view(-1,1).float()
                targets = targets.view(-1).float()
                loss = criterion(outputs.view(-1), targets)
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)
        
        history['val_loss'].append(avg_val_loss)
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, VAL Loss: {avg_val_loss:.4f}')
        #print(f'Epoch [{epoch+1}/{num_epochs}], VAL Loss: {avg_val_loss:.4f}')
        print(f"$$训练集准确率 accuracy: {acc_train}",f"$$训练集精确率 precision: {pre_train}",f"$$训练集召回率 recall: {rec_train}",f"$$训练集 F1 评分 f1_score: {f1_train}")
        


        # 保存训练记录
        historys.append(history)
        

        # 计算、保存测试结果
        model.eval()
        with torch.no_grad():
            pred_test = model(torch.tensor(epoch_test, dtype=torch.float32).to(device))
            
            pred_test = (torch.sigmoid(pred_test) > 0.5).cpu().numpy().astype(int)


        # 保存预测结果和真实结果
        test_pred.append(pred_test)
        test_real.append(label_test)

        # 计算准确率，精确率，召回率，F1评分
        acc = accuracy_score(label_test, pred_test)
        pre = precision_score(label_test, pred_test, average='macro')
        rec = recall_score(label_test, pred_test, average='macro')
        f1 = f1_score(label_test, pred_test, average='macro')
        
        accuracy.append(acc)
        precision.append(pre)
        recall.append(rec)
        f1score.append(f1)
        
        print(f"@@测试集准确率 accuracy: {acc}",f"@@测试集精确率 precision: {pre}",f"@@测试集召回率 recall: {rec}",f"@@测试集 F1 评分 f1_score: {f1}")
        

# 将每一折 history 中误差结果保存（训练集和测试集，用于反映训练过程）    
loss_train = []
loss_val = []
for history_s in historys:
    loss_val.append(history_s['val_loss'])
    loss_train.append(history_s['train_loss'])




fold号: 1


C:\Users\FJQ\AppData\Local\Temp\ipykernel_24736\3703804947.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = TensorDataset(torch.tensor(epoch_train, dtype=torch.float32), torch.tensor(label_train, dtype=torch.long))
C:\Users\FJQ\AppData\Local\Temp\ipykernel_24736\3703804947.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_dataset = TensorDataset(torch.tensor(epoch_val, dtype=torch.float32), torch.tensor(label_val, dtype=torch.long))


开始训练!!
Epoch [1/100], Train Loss: 0.6847, VAL Loss: 0.6711
$$训练集准确率 accuracy: 0.601546557834819 $$训练集精确率 precision: 0.7065548580891803 $$训练集召回率 recall: 0.609232793031869 $$训练集 F1 评分 f1_score: 0.5521240307938723
@@测试集准确率 accuracy: 0.6375579598145286 @@测试集精确率 precision: 0.7270673467819249 @@测试集召回率 recall: 0.6341121941896024 @@测试集 F1 评分 f1_score: 0.5946513313229072
Epoch [2/100], Train Loss: 0.6626, VAL Loss: 0.6603
$$训练集准确率 accuracy: 0.6583610782944904 $$训练集精确率 precision: 0.7372329821331409 $$训练集召回率 recall: 0.6645966979224638 $$训练集 F1 评分 f1_score: 0.6324090344477555
@@测试集准确率 accuracy: 0.6506955177743431 @@测试集精确率 precision: 0.7437460468058191 @@测试集召回率 recall: 0.6473098241590214 @@测试集 F1 评分 f1_score: 0.610863317010858
Epoch [3/100], Train Loss: 0.6408, VAL Loss: 0.6530
$$训练集准确率 accuracy: 0.682633444313178 $$训练集精确率 precision: 0.7510532950167752 $$训练集召回率 recall: 0.688288626932376 $$训练集 F1 评分 f1_score: 0.6633946110238184
@@测试集准确率 accuracy: 0.6205564142194745 @@测试集精确率 precision: 0.728104803900

C:\Users\FJQ\AppData\Local\Temp\ipykernel_24736\3703804947.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = TensorDataset(torch.tensor(epoch_train, dtype=torch.float32), torch.tensor(label_train, dtype=torch.long))
C:\Users\FJQ\AppData\Local\Temp\ipykernel_24736\3703804947.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_dataset = TensorDataset(torch.tensor(epoch_val, dtype=torch.float32), torch.tensor(label_val, dtype=torch.long))


开始训练!!
Epoch [1/100], Train Loss: 0.6908, VAL Loss: 0.6855
$$训练集准确率 accuracy: 0.5057458919557513 $$训练集精确率 precision: 0.45959256355524014 $$训练集召回率 recall: 0.4980448146364007 $$训练集 F1 评分 f1_score: 0.34477801379450057
@@测试集准确率 accuracy: 0.5015455950540958 @@测试集精确率 precision: 0.5212270341207349 @@测试集召回率 recall: 0.5015455950540958 @@测试集 F1 评分 f1_score: 0.35114245666081273
Epoch [2/100], Train Loss: 0.6715, VAL Loss: 0.6726
$$训练集准确率 accuracy: 0.6707120610031146 $$训练集精确率 precision: 0.7414804865437539 $$训练集召回率 recall: 0.6663491902771416 $$训练集 F1 评分 f1_score: 0.6411845931547924
@@测试集准确率 accuracy: 0.7967542503863988 @@测试集精确率 precision: 0.7997799142824047 @@测试集召回率 recall: 0.7967542503863988 @@测试集 F1 评分 f1_score: 0.796240115769804
Epoch [3/100], Train Loss: 0.6515, VAL Loss: 0.6659
$$训练集准确率 accuracy: 0.774997315003759 $$训练集精确率 precision: 0.7853211169540485 $$训练集召回率 recall: 0.7764625550770579 $$训练集 F1 评分 f1_score: 0.773527899623069
@@测试集准确率 accuracy: 0.7171561051004637 @@测试集精确率 precision: 0.7605334

...

开始训练!!
Epoch [1/100], Train Loss: 0.6794, VAL Loss: 0.6718
$$训练集准确率 accuracy: 0.5052620274914089 $$训练集精确率 precision: 0.4528011204481793 $$训练集召回率 recall: 0.49939364191821745 $$训练集 F1 评分 f1_score: 0.3379732096976705
@@测试集准确率 accuracy: 0.5444702242846094 @@测试集精确率 precision: 0.6377437617949255 @@测试集召回率 recall: 0.5157756206742523 @@测试集 F1 评分 f1_score: 0.3916849382361046
Epoch [2/100], Train Loss: 0.6610, VAL Loss: 0.6538
$$训练集准确率 accuracy: 0.7207903780068728 $$训练集精确率 precision: 0.7353782306439931 $$训练集召回率 recall: 0.7192746161750574 $$训练集 F1 评分 f1_score: 0.71546452292288
@@测试集准确率 accuracy: 0.7262180974477959 @@测试集精确率 precision: 0.7256785868160276 @@测试集召回率 recall: 0.7264254254302333 @@测试集 F1 评分 f1_score: 0.7257573215740785
Epoch [3/100], Train Loss: 0.6422, VAL Loss: 0.6456
$$训练集准确率 accuracy: 0.7374355670103093 $$训练集精确率 precision: 0.7486726429461443 $$训练集召回率 recall: 0.7386562296614798 $$训练集 F1 评分 f1_score: 0.7350641572693859
@@测试集准确率 accuracy: 0.6867749419953596 @@测试集精确率 precision: 0.74688385

C:\Users\FJQ\AppData\Local\Temp\ipykernel_24736\3703804947.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = TensorDataset(torch.tensor(epoch_train, dtype=torch.float32), torch.tensor(label_train, dtype=torch.long))
C:\Users\FJQ\AppData\Local\Temp\ipykernel_24736\3703804947.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_dataset = TensorDataset(torch.tensor(epoch_val, dtype=torch.float32), torch.tensor(label_val, dtype=torch.long))


开始训练!!
Epoch [1/100], Train Loss: 0.6874, VAL Loss: 0.6713
$$训练集准确率 accuracy: 0.5098797250859106 $$训练集精确率 precision: 0.5195986376744895 $$训练集召回率 recall: 0.5030769496956548 $$训练集 F1 评分 f1_score: 0.37355419146509544
@@测试集准确率 accuracy: 0.5931941221964424 @@测试集精确率 precision: 0.6423847554995097 @@测试集召回率 recall: 0.5731858381586149 @@测试集 F1 评分 f1_score: 0.5252985912189893
Epoch [2/100], Train Loss: 0.6607, VAL Loss: 0.6526
$$训练集准确率 accuracy: 0.7825386597938144 $$训练集精确率 precision: 0.7836901795264075 $$训练集召回率 recall: 0.7829859402521273 $$训练集 F1 评分 f1_score: 0.7824635773752107
@@测试集准确率 accuracy: 0.7122969837587007 @@测试集精确率 precision: 0.7589164227678398 @@测试集召回率 recall: 0.7238390516521188 @@测试集 F1 评分 f1_score: 0.7050262473629986
Epoch [3/100], Train Loss: 0.6366, VAL Loss: 0.6351
$$训练集准确率 accuracy: 0.7343213058419243 $$训练集精确率 precision: 0.7675209014117621 $$训练集召回率 recall: 0.7368830476744146 $$训练集 F1 评分 f1_score: 0.7271662193235187
@@测试集准确率 accuracy: 0.6604795050270689 @@测试集精确率 precision: 0.750212

C:\Users\FJQ\AppData\Local\Temp\ipykernel_24736\3703804947.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = TensorDataset(torch.tensor(epoch_train, dtype=torch.float32), torch.tensor(label_train, dtype=torch.long))
C:\Users\FJQ\AppData\Local\Temp\ipykernel_24736\3703804947.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_dataset = TensorDataset(torch.tensor(epoch_val, dtype=torch.float32), torch.tensor(label_val, dtype=torch.long))


开始训练!!
Epoch [1/100], Train Loss: 0.6801, VAL Loss: 0.6758
$$训练集准确率 accuracy: 0.7265893470790378 $$训练集精确率 precision: 0.7269739460082683 $$训练集召回率 recall: 0.7259756897669705 $$训练集 F1 评分 f1_score: 0.7260373229894352
@@测试集准确率 accuracy: 0.7587006960556845 @@测试集精确率 precision: 0.7653383810787604 @@测试集召回率 recall: 0.7603841513625309 @@测试集 F1 评分 f1_score: 0.7578860945467116
Epoch [2/100], Train Loss: 0.6592, VAL Loss: 0.6551
$$训练集准确率 accuracy: 0.7636383161512027 $$训练集精确率 precision: 0.7743138043722806 $$训练集召回率 recall: 0.7655806638245213 $$训练集 F1 评分 f1_score: 0.7621321034220416
@@测试集准确率 accuracy: 0.765661252900232 @@测试集精确率 precision: 0.7892459158281944 @@测试集召回率 recall: 0.7687886593002546 @@测试集 F1 评分 f1_score: 0.7621090725824815
Epoch [3/100], Train Loss: 0.6369, VAL Loss: 0.6359
$$训练集准确率 accuracy: 0.7468857388316151 $$训练集精确率 precision: 0.7735734418453288 $$训练集召回率 recall: 0.74999165871033 $$训练集 F1 评分 f1_score: 0.7420499190958451
@@测试集准确率 accuracy: 0.7587006960556845 @@测试集精确率 precision: 0.7854163190

In [7]:
# 假设 accuracy 列表包含了 1000 个准确率值
# 每一折包含 100 个 epoch
num_folds = 10
epochs_per_fold = 100

# 打印每一折的最大准确率和对应的epoch
for fold in range(num_folds):
    # 获取当前折的准确率列表
    start_idx = fold * epochs_per_fold
    end_idx = (fold + 1) * epochs_per_fold
    fold_accuracy = accuracy[start_idx:end_idx]

    # 找到最大准确率和对应的epoch
    max_acc = max(fold_accuracy)
    max_epoch = fold_accuracy.index(max_acc) + 1  # +1 因为索引从0开始

    # 打印结果
    print(f'第{fold + 1}折准确率最高为{max_acc:.4f}，对应的epoch为{max_epoch}')

第1折准确率最高为0.8972，对应的epoch为74
第2折准确率最高为0.9196，对应的epoch为99
第3折准确率最高为0.8988，对应的epoch为99
第4折准确率最高为0.8855，对应的epoch为91
第5折准确率最高为0.8886，对应的epoch为78
第6折准确率最高为0.9296，对应的epoch为87
第7折准确率最高为0.9002，对应的epoch为82
第8折准确率最高为0.8631，对应的epoch为86
第9折准确率最高为0.8786，对应的epoch为82
第10折准确率最高为0.9002，对应的epoch为79
